In [3]:
"""deluca.agents._mem"""
from numbers import Real
from typing import Callable

import jax
import jax.numpy as jnp
import numpy as np
from jax import grad
from jax import jit

from deluca.agents._lqr import LQR
from deluca.agents.core import Agent

from deluca.utils.optimR import optimROBD

/Users/ayushalag/anaconda3/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [35]:
def idx(B):
    return np.where(B.any(axis=1))[0].astype(int) #return indices of non-zero rows

In [54]:
'''Test idx'''
B = np.array([[1,2,3], [0, 0, 0], [0,1,0]])
ks = idx(B)
print(ks)
print("works!")

[0 2]
works!


In [37]:
def defineP(ks):
    ps = np.ndarray((len(ks), 1))
    ps[0] = ks[0]
    for i in range(1,len(ks)):
        ps[i] = ks[i] - ks[i-1]
    return ps.astype(int), int(np.amax(ps))

In [38]:
''' Test defineP '''
ps, p = defineP(idx(B))
print(ps, p)

[[0]
 [1]
 [1]] 1


In [52]:
def defineCs(A, B, p, d, ps, ks):
    Cs = np.ndarray((p, d, d))

    for i in range(1, p+1):
        a_identity = A[ks.astype(int)] #check slicing
        C = np.ndarray((d, d))
        for j in range(1, d+1):
            if i <= ps[j-1]:
                C[:, j-1] = a_identity[:, ks[j-1]+1-i]
            else:
                C[:, j-1] = np.zeros(d)
        Cs[i-1] = C
    return Cs

In [56]:
''' Testing defineCs '''
A = np.array([[1,2,3], [1,3,4], [1,3,5]])
d = 2
defineCs(A, B, p, d, ps, ks)

array([[[0., 3.],
        [0., 5.]]])

In [79]:
def hitFunc(qs, ps, ks, t):
    def func(y):
        masterSum = 0
        for i in range(d):
            littleSum = 0
            for j in range(1, int(ps[i-1])):
                littleSum += qs[t+j]
            print(littleSum)
            masterSum += (littleSum * (y[ks[i]] ** 2))
        return masterSum/2

    return func

In [81]:
print(np.ones(d))
print(ps)
hitFunc(np.ones(d), ps, ks, 2)(np.array([1,2,3]))

[1. 1.]
[[0]
 [1]
 [1]]
0
0


0.0